# 🔧 Debug Drill: The High-Accuracy Flop

**Scenario:**
Your colleague deployed a churn model to production. "It has 89% accuracy!" they said.

But after a month, the retention team is frustrated.

"We called 500 customers your model flagged as at-risk," they say. "Only 30 of them actually churned. That's worse than random!"

**Your Task:**
1. Run the notebook and find why high accuracy doesn't mean good performance
2. Identify the correct metric for this use case
3. Fix the evaluation
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Load data
try:
    df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/streamcart_customers.csv')
except:
    df = pd.read_csv('../../data/streamcart_customers.csv')

print(f"Loaded {len(df):,} customers")
print(f"Churn rate: {df['churn_30d'].mean():.1%}")

In [ ]:
# ===== COLLEAGUE'S CODE =====

features = ['tenure_months', 'logins_last_30d', 'support_tickets_last_30d', 'nps_score']

X = df[features].fillna(0)
y = df['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Colleague's evaluation (WRONG METRIC FOR THE USE CASE)
y_pred = model.predict(X_test)  # Binary predictions
accuracy = accuracy_score(y_test, y_pred)

print(f"\n🎉 Model Performance")
print(f"Accuracy: {accuracy:.1%}")
print(f"\n'Ship it!' - Your colleague")

In [ ]:
# What the retention team actually did:
# They called the top 500 customers the model predicted as "churn"

y_proba = model.predict_proba(X_test)[:, 1]
top_500_idx = np.argsort(y_proba)[-500:]

# How many actually churned?
actual_churners_in_top_500 = y_test.iloc[top_500_idx].sum()
precision_at_500 = y_test.iloc[top_500_idx].mean()

print(f"\n😤 Retention Team's Reality Check")
print(f"Customers called: 500")
print(f"Actual churners found: {actual_churners_in_top_500}")
print(f"Precision@500: {precision_at_500:.1%}")
print(f"\n'This is barely better than random ({y_test.mean():.1%})!'")

---

## Your Investigation

### Step 1: Why is accuracy misleading?

In [ ]:
# TODO: Create a "dummy" model that predicts NO ONE will churn
# What's its accuracy?

dummy_predictions = np.zeros(len(y_test))  # Predict 0 for everyone
dummy_accuracy = accuracy_score(y_test, dummy_predictions)

print(f"Dummy model accuracy: {dummy_accuracy:.1%}")
print(f"Our model accuracy: {accuracy:.1%}")
print(f"\n❓ Why is the dummy model almost as good?")

In [ ]:
# What do you notice? Write your diagnosis:

diagnosis = """
YOUR DIAGNOSIS:

Why is accuracy misleading for this problem?

What metric should we use instead, given that the team can only call 500 customers?

"""
print(diagnosis)

### Step 2: Use the right metric

In [ ]:
# TODO: Calculate the correct metrics for this use case
#
# Given: Team can call 500 customers per week
# Goal: Maximize churners found in those 500 calls
# 
# What metric captures this? Precision@K!

def precision_at_k(y_true, y_proba, k):
    """Calculate precision in top K predictions."""
    top_k_idx = np.argsort(y_proba)[-k:]
    return y_true.iloc[top_k_idx].mean()

# Calculate for different K values
# ...

# Compare to random baseline
# ...

In [ ]:
# Now train a better model optimizing for the RIGHT goal

# Try different model configurations
# Track Precision@500, not accuracy
# ...

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# A good model should have Precision@500 at least 1.5x the baseline
baseline = y_test.mean()
# lift = your_precision_at_500 / baseline

# assert lift > 1.5, f"Lift is only {lift:.1f}x - model isn't useful!"
# print(f"✓ Precision@500 lift: {lift:.1f}x baseline")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: Wrong Metric Bug

### What happened:
- (Your answer)

### Root cause:
- (Your answer - explain why accuracy is wrong for imbalanced data)

### How to prevent:
- (Your answer - mention business constraints → metric choice)
"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lesson:** Always choose metrics based on business constraints. High accuracy means nothing if the model doesn't help you make better decisions.